# Advanced Costing Features

The purpose of this tutorial is to introduce advanced features available through the REE Costing Framework in PrOMMiS. To learn the basics of the REE Costing Framework, please refer to the [Basic Costing Features](costing_basic_features-solution.ipynb) tutorial.

## Introduction

This notebook demonstrates advanced features of the REE Costing Framework. The methods described here complement the basic costing framework to add complexity and enabled more detailed cost modeling.

<div class="alert alert-block alert-info">
<b>Note:</b>
Calling the function `build_process_costs` on a costing block more than once is not allowed, due to the search algorithm that aggregates process and stream costs. Therefore, a new costing block will be created for each use case in this notebook where needed.
</div>

## Learning Objectives

The tutorial will take users through the following:

1. Importing the required tools from PrOMMiS and related repositories
2. Importing and using cost models from WaterTAP
3. Writing and importing custom, user-defined cost models
4. Calculating net present value and tax estimation
5. Standalone method for predicting process costs using economy of numbers
6. Standalone method for estimating cost bounds for general process types

Useful Links:
* Public GitHub Repository: https://github.com/prommis/prommis/tree/main
* REE Costing Module Code: https://github.com/prommis/prommis/blob/main/src/prommis/uky/costing/ree_plant_capcost.py

## Problem Statement

This tutorial will demonstrate costing a membrane nanofiltration process separating lithium and magnesium ions from brine in order to introduce advanced costing features:

![nf_ui.png](./nf_ui.png)

# 1 Import the necessary tools

First, import the required Python, Pyomo, IDAES, and PrOMMiS packages. These will be implemented at various stages of the tutorial.

In [1]:
# import pytest
import pytest

# Pyomo packages
from pyomo.environ import (
    ConcreteModel,
    TransformationFactory,
    Constraint,
    Var,
    Param,
    units as pyunits,
    value,
)
from pyomo.network import Arc

# IDAES packages
from idaes.core import FlowsheetBlock, UnitModelBlock, UnitModelCostingBlock
from idaes.core.solvers import get_solver
from idaes.models.unit_models import Feed, Product


# PrOMMiS packages
from prommis.uky.costing.ree_plant_capcost import QGESSCosting, QGESSCostingData

# WaterTAP packages
from prommis.nanofiltration.nf_brine import (
    define_feed_composition,
    initialize,
    solve_model,
)
from watertap.core.solvers import (
    get_solver as get_watertap_solver,
)  # WaterTAP has its own solver

from watertap.property_models.multicomp_aq_sol_prop_pack import (
    MCASParameterBlock,
)
from watertap.unit_models.nanofiltration_DSPMDE_0D import NanofiltrationDSPMDE0D
from watertap.unit_models.pressure_changer import Pump

# 2 Build Process Model Flowsheet

The REE Costing Framework supports integration with costing methods imported from [WaterTAP](https://watertap.readthedocs.io/en/stable/), an external package for modeling water treatment operations such as aqueous membrane processes involving REE components. Cost accounts imported from the built-in dictionary may be attached to either unit model blocks or generic `UnitModelBlock` objects. In contrast, WaterTAP costing methods look for the unit model class, i.e. to import costing for a WaterTAP model the flowsheet needs to contain that WaterTAP model before calling `build_process_costs`. The costing framework handles passing unit model references, importing costing methods, and extracting the necessary capital and operating cost components. On the flowsheet side, users only need to define their unit models and pass them to the costing method.

## 2.1 Build Flowsheet

For this example, we will build a flowsheet containing a WaterTAP unit model. We'll see that the `build_process_costs` method will automatically extract the relevant costing objects.

To begin, let's create the initial flowsheet:

In [2]:
# Create a Concrete Model as the top level object
m = ConcreteModel()

# Add a flowsheet object to the model
m.fs = FlowsheetBlock(dynamic=False)

## 2.2 Add Unit Models

We will use the `NanofiltrationDSPMDE0D` unit model, a time-dependent model implementing the Donnan Steric Pore Model with Dielectric Exclusion (DSPM-DE) for nanofiltration. More information on this model may be found in the [NanofiltrationDSPMDE0D documentation](https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/nanofiltration_dspmde_0D.html). The specific model is the [Nanofiltration Brine](https://github.com/prommis/prommis/tree/main/src/prommis/nanofiltration) model, which exists in the PrOMMiS repository:

In [3]:
# WaterTAP has its own solver
watertap_solver = get_watertap_solver()

# Define the property model - feed composition method is imported from the NF flowsheet
default = define_feed_composition()
m.fs.properties = MCASParameterBlock(**default)

# Add the feed and product streams
m.fs.feed = Feed(property_package=m.fs.properties)
m.fs.permeate = Product(property_package=m.fs.properties)
m.fs.retentate = Product(property_package=m.fs.properties)

# Define unit models; note that the NF model needs to be named "unit" for the imported methods to work
m.fs.pump = Pump(property_package=m.fs.properties)
m.fs.unit = NanofiltrationDSPMDE0D(property_package=m.fs.properties)

# Connect the streams and blocks
m.fs.feed_to_pump = Arc(source=m.fs.feed.outlet, destination=m.fs.pump.inlet)
m.fs.pump_to_nf = Arc(source=m.fs.pump.outlet, destination=m.fs.unit.inlet)
m.fs.nf_to_permeate = Arc(source=m.fs.unit.permeate, destination=m.fs.permeate.inlet)
m.fs.nf_to_retentate = Arc(source=m.fs.unit.retentate, destination=m.fs.retentate.inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

# The methods below are imported from the NF flowsheet

# Initialize
initialize(m, watertap_solver)

# Solve
solve_model(m, watertap_solver)

# Re-solve
solve_model(m, watertap_solver)
print("Model Solved")

('Liq', 'H2O') flow_mol_phase_comp scaling factor = 0.1
('Liq', 'Li_+') flow_mol_phase_comp scaling factor = 10
('Liq', 'Mg_2+') flow_mol_phase_comp scaling factor = 10
('Liq', 'Cl_-') flow_mol_phase_comp scaling factor = 1
Cl_- adjusted: fs.feed.properties[0.0].flow_mol_phase_comp['Liq',Cl_-] was adjusted from 4.843574775634025 and fixed to 0.9219738584555871. Electroneutrality satisfied for fs.feed.properties[0.0]. Balance Result = 1.1368683772161603e-13
2025-08-19 08:01:59 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.area
2025-08-19 08:01:59 [INFO] idaes.init.fs.feed: Initialization Complete.
2025-08-19 08:01:59 [INFO] idaes.init.fs.pump.control_volume: Initialization Complete
2025-08-19 08:01:59 [INFO] idaes.init.fs.pump: Initialization Complete: optimal - Optimal Solution Found
2025-08-19 08:02:01 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found
2025-08-19 08:02:01 [INFO] idaes.init.fs.permeate: Initialization Complete.


Let's explore the unit we just built to check for costing objects:

In [4]:
for i in m.fs.unit.component_data_objects(descend_into=True):
    if "cost" in str(i):
        print(i)
print("Search complete")

Search complete


As expected, there are no costing objects on the model yet. We don't need to add this ourselves, as the REE Costing Framework will call the correct objects from WaterTAP automatically when we build the plant costs.

## 2.3 Build Process Costs

Next, let's define some process-wide parameters such as the product and resource rates and build process costs. In this case, the costing methods expects pure and mixed product rates; in this scenario, there is no saleable product since the retentate requires further processing, so we'll use the retentate flow as a "treated stream" product and tell the costing that it can't be sold (price = 0). There is still a waste disposal cost for the permeate and cost of chemicals per gallon (arbitrarily chosen for this example), which are included below:

In [5]:
# Define resources
m.fs.water = Var([0], initialize=1000, units=pyunits.gallon / pyunits.hr)
m.fs.water.fix()

m.fs.chemicals = Var([0], initialize=20, units=pyunits.gallon / pyunits.hr)
m.fs.chemicals.fix()

m.fs.waste = Var([0], initialize=20, units=pyunits.gallon / pyunits.hr)
m.fs.waste_constraint = Constraint(
    expr=(
        m.fs.waste[0]
        == pyunits.convert(
            m.fs.unit.feed_side.properties_out[0].flow_vol_phase["Liq"],
            to_units=pyunits.gal / pyunits.hr,
        )
    )
)

# Define product rate - required for costing, but given price = 0 in this scenario (not a saleable product)
m.fs.product = Var([0], initialize=20, units=pyunits.gallon / pyunits.hr)
m.fs.product_constraint = Constraint(
    expr=(
        m.fs.product[0]
        == pyunits.convert(
            m.fs.unit.permeate_side[0, 1].flow_vol_phase["Liq"],
            to_units=pyunits.gal / pyunits.hr,
        )
    )
)

# Attach a flowsheet costing block
m.fs.costing_1 = QGESSCosting()

# Build process costs
m.fs.costing_1.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    # treated stream is not saleable, so pass dummy inputs
    pure_product_output_rates={"treated_stream": m.fs.product[0]},
    mixed_product_output_rates={"treated_stream": m.fs.product[0]},
    sale_prices={"treated_stream": 0 * pyunits.USD_2021 / pyunits.gal},
    variable_OM=True,
    resources=[
        "water",
        "chemicals",
        "waste",
    ],
    rates=[m.fs.water, m.fs.chemicals, m.fs.waste],
    prices={
        "chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon,
        "waste": 0.35 * pyunits.USD_2021 / pyunits.gallon,
    },
    watertap_blocks=[
        m.fs.unit,
    ],
    CE_index_year="2021",
)

QGESSCostingData.costing_initialization(m.fs.costing_1)
QGESSCostingData.initialize_fixed_OM_costs(m.fs.costing_1)
QGESSCostingData.initialize_variable_OM_costs(m.fs.costing_1)

# Solve and display results
solver = get_solver()
solver.solve(m, tee=False)
# m.fs.costing_1.report() # uncomment to view cost results
assert m.fs.costing_1.total_BEC.value == pytest.approx(0.0035218, rel=1e-4)
assert m.fs.costing_1.total_installation_cost.value == pytest.approx(
    0.0069380, rel=1e-4
)
assert m.fs.costing_1.total_plant_cost.value == pytest.approx(0.010460, rel=1e-4)
assert m.fs.costing_1.total_fixed_OM_cost.value == pytest.approx(5.1756, rel=1e-4)
assert m.fs.costing_1.total_sales_revenue.value == pytest.approx(0, abs=1e-8)
assert m.fs.costing_1.total_variable_OM_cost[0].value == pytest.approx(3.9968, rel=1e-4)
assert m.fs.costing_1.plant_overhead_cost[0].value == pytest.approx(1.0351, rel=1e-4)

assert m.fs.costing_1.watertap_fixed_costs.value == pytest.approx(0.000352180, rel=1e-4)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.


Checking some of the model objects:

In [6]:
for i in m.fs.costing_1.component_data_objects(Var, descend_into=True):
    if "watertap" in str(i) or "BEC" in str(i):
        print(i.name, "has a value of ", value(i))

print()

for i in m.fs.costing_1.component_data_objects(Constraint, descend_into=True):
    if "watertap" in str(i.expr) or "BEC" in str(i.expr):
        print(i.expr)
        print()

fs.costing_1.total_BEC has a value of  0.0035218040126026163
fs.costing_1.watertap_fixed_costs has a value of  0.00035218040126027783

fs.costing_1.total_BEC  ==  1.1739346708671863e-06*(MUSD_2021/USD_2018)*fs.unit.costing.capital_cost

fs.costing_1.total_installation_cost  ==  (fs.costing_1.Lang_factor - 1)*fs.costing_1.total_BEC

fs.costing_1.total_plant_cost  ==  fs.costing_1.total_BEC + fs.costing_1.total_installation_cost + fs.costing_1.other_plant_costs

fs.costing_1.watertap_fixed_costs  ==  1.1739346708671863e-06*(MUSD_2021/USD_2018)*fs.unit.costing.fixed_operating_cost

fs.costing_1.total_fixed_OM_cost  ==  fs.costing_1.annual_labor_cost + fs.costing_1.maintenance_and_material_cost + fs.costing_1.quality_assurance_and_control_cost + fs.costing_1.admin_and_support_labor_cost + fs.costing_1.sales_patenting_and_research_cost + fs.costing_1.property_taxes_and_insurance_cost + fs.costing_1.other_fixed_costs + fs.costing_1.watertap_fixed_costs + fs.costing_1.custom_fixed_costs



We see that the model has a variable called `watertap_fixed_costs`. This is a placeholder variable to catch operating costs returned by WaterTAP. There is no `watertap_variable_costs` variable, as no current WaterTAP models contains variable operating costs. The expressions for `total_BEC` and `watertap_fixed_costs` expressions each include terms for the `NanofiltrationDSPMDE0D` costs.

Multiple WaterTAP models may be passed to the argument `watertap_blocks`, and the REE Costing Framework will automatically extract all capital and operating cost components.

# 3 User-Defined Custom Costing

Suppose we want to add costing for a custom unit model to the flowsheet modeling a vessel to wash and extract the permeate stream. For this example, we'll assume that this makes the stream saleable. We know the volume of the vessel and water injection rate.

The `UnitModelCostingBlock` takes several arguments, including `costing_method` which points to an external class containing a function that defines the cost model. The argument `costing_method_arguments` aligns with the inputs of that function. Users may write their own functions, which is demonstrated below.

## 3.1 Writing a Custom Cost Model

Suppose we have written our own costing model for the vessel. The equations below have been arbitrarily chosen for demonstrative purposes.

The capital cost (in dollars) assumes a six-tenths power law based on vessel volume:
```
Capital_Cost = Reference_Cost * (Volume / Reference_Volume) ** 0.6
```

Fixed operating costs (in dollars per year) are a set percentage of the capital cost:
```
Fixed_Costs = 0.05 * Capital_Cost / year
```

Variable operating costs (in dollars per year) are a function of the water injection rate:
```
Variable_Operating_Costs = ($.00190/gal) * Water_Injection_Rate
```

We'll add arguments for the vessel volume, material, water injection rate, and number of units. We'll also allow a string for the cost year, which will set the currency units for the cost variables.

The custom model is formulated below:

In [7]:
# Import necessary IDAES functions to declare a new process block class
from idaes.core import declare_process_block_class, FlowsheetCostingBlockData

# Import a Pyomo function for variable bound ranges
from pyomo.environ import NonNegativeReals


@declare_process_block_class(
    "CustomCosting"
)  # a new class should always open with a declaration decorator
class CustomCostingData(
    FlowsheetCostingBlockData
):  # the class inherits all properties of the IDAES FlowsheetCostingBlock class
    # Register custom currency units used in the custom costing model
    pyunits.load_definitions_from_strings(
        [
            "USD_custom = 500/700 * USD_CE500",
        ]
    )  # CEPCI = 700, USD_CE500 is a placeholder that cancels out in the unit conversion

    # Define global parameters - base cost year and period
    def build_global_params(self):

        # Set the base year for all costs
        self.base_currency = pyunits.USD_2022
        # Set a base period for all operating costs
        self.base_period = pyunits.year

    # Custom costing method
    def cost_custom_vessel(
        blk,  # when the costing block is built, blk will be the costing block itself
        volume_per_unit=1000 * pyunits.m**3,
        material="carbonsteel",
        water_injection_rate_per_unit=1 * pyunits.m**3 / pyunits.s,
        number_of_units=1,
    ):

        # Make parameter for number of units
        blk.number_of_units = Param(
            initialize=number_of_units, mutable=True, units=pyunits.dimensionless
        )

        # Define the bare erected cost (BEC) per unit

        material_factor_dict = (
            {  # material factors for each valid choice of shell material
                "carbonsteel": 1,
                "stainlessteel": 1.5,
                "aluminum": 2,
            }
        )

        blk.capital_cost_per_unit = Var(
            initialize=1000,
            units=blk.costing_package.base_currency,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def capital_cost_per_unit_eq(blk):
            # cost equation CAPITAL_COST = REF_COST * (VOLUME / REF_VOLUME)**0.6
            ref_cost = (
                10000 * pyunits.USD_custom
            )  # reference cost is in reference currency units
            ref_volume = 1000 * pyunits.m**3
            return blk.capital_cost_per_unit == pyunits.convert(
                material_factor_dict[material]
                * ref_cost
                * (volume_per_unit / ref_volume) ** 0.6,
                to_units=blk.costing_package.base_currency,  # convert to costing block base currency
            )

        # Create a variable capital_cost that the REE Costing Framework can look for
        blk.capital_cost = Var(
            initialize=1000,
            units=blk.costing_package.base_currency,  # define in costing block base currency
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def capital_cost_constraint(blk):
            return blk.capital_cost == blk.capital_cost_per_unit * blk.number_of_units

        # Define the fixed costs

        # Create a variable fixed_operating_cost that the REE Costing Framework can look for
        blk.fixed_operating_cost = Var(
            initialize=1000,
            units=blk.costing_package.base_currency
            / blk.costing_package.base_period,  # define in costing block base currency
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        # Set fixed opex = 5% of capex
        # This is an arbitrary choice for this example, and not a general rule
        # In practice users should create Param and Var as needed for their model
        blk.fixed_opex_coefficient = Param(
            initialize=0.05, mutable=True, units=pyunits.dimensionless
        )

        @blk.Constraint()
        def fixed_operating_cost_constraint(blk):
            return blk.fixed_operating_cost == pyunits.convert(
                blk.fixed_opex_coefficient * blk.capital_cost / pyunits.year,
                to_units=blk.costing_package.base_currency
                / blk.costing_package.base_period,
            )

        # Define the variable costs

        blk.variable_operating_cost_per_unit = Var(
            initialize=1000,
            units=blk.costing_package.base_currency
            / blk.costing_package.base_period,  # define in costing block base currency / time
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        # Set variable opex = $0.0019 per gallon of water injected using the reference year (USD_custom)
        # This is an arbitrary choice for this example, and not a general rule
        blk.variable_opex_price = Param(
            initialize=0.00190,
            units=pyunits.USD_custom
            / pyunits.gal,  # define in reference currency units
            mutable=True,
        )

        @blk.Constraint()
        def variable_operating_cost_per_unit_eq(blk):
            return blk.variable_operating_cost_per_unit == pyunits.convert(
                blk.variable_opex_price * water_injection_rate_per_unit,
                to_units=blk.costing_package.base_currency
                / blk.costing_package.base_period,  # define in costing block base currency / time
            )

        # Create a variable variable_operating_cost that the REE Costing Framework can look for
        blk.variable_operating_cost = Var(
            initialize=1000,
            units=blk.costing_package.base_currency
            / blk.costing_package.base_period,  # define in costing block base currency / time
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def variable_operating_cost_constraint(blk):
            return (
                blk.variable_operating_cost
                == blk.variable_operating_cost_per_unit * blk.number_of_units
            )

There are a few important details in the model above. First, the decorator `@declare_process_block_class()` tells the Python interpreter to register the new class as an importable object with the name "CustomCosting". The class inherits all properties of the `FlowsheetCostingBlock` class by calling its corresponding data class.

The first line inside the custom class defines a new cost unit `USD_custom` with a Chemical Engineering Plant Cost Index (CEPCI) value of 700. `USD_CE500` is a reference unit container enabling conversion between currency units. The method `build_global_params` defines the base cost year and period for the model. Since the class inherits all properties of the `FlowsheetCostingBlockData` class, the final model will return cost variables in units of `USD_2021` and any time units will be `year` or `a` (annum, the two are equivalent in Pyomo unit container syntax).

Then, the method `cost_custom_vessel` defines all cost equations. Each of the three cost variables (capital, fixed operating, variable operating) is built by defining a cost per unit with appropriate inputs, and then a total cost based on the number of units. The naming convention is important, since the REE Costing Framework will look for variables named `capital_cost`, `fixed_operating_cost`, and `variable_operating_cost` when building the process cost equations.

Finally, note that the currency units are set as `costing_package.base_currency` everywhere in the custom model. The object `costing_package` is the main flowsheet costing class set by the user, so the model will automatically use those default cost units. The examples below will demonstrate this further.

## 3.2 Create a Costing Block

Let's create a new flowsheet using our custom costing model. We'll use a `UnitModelBlock` for the vessel itself and attach it to the custom class object. The vessel should be large enough to hold 3 days worth of product flow, and the water injection rate will be 5% of the product flow (these are arbitrarily chosen for this example):

In [8]:
# Define vessel model
m.fs.custom_vessel = UnitModelBlock()
m.fs.custom_vessel.volume = Var(initialize=5000, units=pyunits.m**3)

m.fs.custom_vessel.volume_constraint = Constraint(
    expr=(
        m.fs.custom_vessel.volume
        == pyunits.convert(m.fs.product[0] * 3 * pyunits.day, to_units=pyunits.m**3)
    )
)

m.fs.custom_vessel.water_injection_rate = Var(
    initialize=0.1, units=pyunits.m**3 / pyunits.s
)

m.fs.custom_vessel.water_injection_rate_constraint = Constraint(
    expr=(
        m.fs.custom_vessel.water_injection_rate
        == pyunits.convert(m.fs.product[0] * 5 / 100, to_units=pyunits.m**3 / pyunits.s)
    )
)

# Attach a flowsheet costing block
m.fs.costing_2 = QGESSCosting()

# Add costing
m.fs.custom_vessel.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing_2,
    costing_method=CustomCostingData.cost_custom_vessel,  # this is the custom method we wrote above
    costing_method_arguments={
        "volume_per_unit": m.fs.custom_vessel.volume,
        "material": "carbonsteel",
        "water_injection_rate_per_unit": m.fs.custom_vessel.water_injection_rate,
        "number_of_units": 1,
        # no CE_index_year argument, since our class doesn't accept this as an argument
        # the cost variables will be returned in units of base_currency = USD_2021
    },
)

# Solve and display results (uncomment line to display)
solver.solve(m, tee=True)
# m.fs.custom_vessel.display() # uncomment to view cost results
assert m.fs.custom_vessel.costing.capital_cost.value == pytest.approx(294.011, rel=1e-4)
assert m.fs.custom_vessel.costing.fixed_operating_cost.value == pytest.approx(
    14.7006, rel=1e-4
)
assert m.fs.custom_vessel.costing.variable_operating_cost.value == pytest.approx(
    8.49301, rel=1e-4
)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resu

The costing block contains the custom model, and cost variables are in units of `MUSD_2022` which match the default units of the custom class, `self.base_currency = pyunits.USD_2022`. Note that the model is not solved yet, so the variables are at their default values.

## 3.3 Build Process Costing

Now, we can build the process costs. Because we defined the capital costing using a `UnitModelCostingBlock`, we see that the custom model is present in the registered cost for the flowsheet:

In [9]:
for block in m.fs.costing_2._registered_unit_costing:
    print(block.name)

fs.custom_vessel.costing


This means that we don't need to pass a special list including the unit, and the extra variables and constraints associated with the custom costing model will be added automatically. The REE Costing Framework will see that our custom model is already registered and will automatically check for `capital_cost`, `fixed_operating_cost`, and `variable_operating_cost` variables.

Note that we still need the product rates to calculate fixed operating costs. 

Let's build the process costs:

In [10]:
# Build process costs
m.fs.costing_2.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    pure_product_output_rates={
        "treated_stream": m.fs.product[0] * 1.05
    },  # includes injected water
    mixed_product_output_rates={
        "treated_stream": m.fs.product[0] * 0
    },  # the product has 100% price realization, and is not considered a mixed product
    sale_prices={"treated_stream": 1 * pyunits.USD_2021 / pyunits.gal},
    variable_OM=True,
    resources=[
        "water",
        "chemicals",
        "waste",
    ],
    rates=[m.fs.water, m.fs.chemicals, m.fs.waste],
    prices={
        "chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon,
        "waste": 0.35 * pyunits.USD_2021 / pyunits.gallon,
    },
    watertap_blocks=[
        m.fs.unit,
    ],
    CE_index_year="2021",
)

In [11]:
QGESSCostingData.costing_initialization(m.fs.costing_2)
QGESSCostingData.initialize_fixed_OM_costs(m.fs.costing_2)
QGESSCostingData.initialize_variable_OM_costs(m.fs.costing_2)

# Solve and display results
solver.solve(m, tee=False)

# m.fs.costing_2.report() # uncomment to view results
assert m.fs.costing_2.total_BEC.value == pytest.approx(0.0038158, rel=1e-4)
assert m.fs.costing_2.total_installation_cost.value == pytest.approx(
    0.0075172, rel=1e-4
)
assert m.fs.costing_2.total_plant_cost.value == pytest.approx(0.011333, rel=1e-4)
assert m.fs.costing_2.total_fixed_OM_cost.value == pytest.approx(5.1761, rel=1e-4)
assert m.fs.costing_2.total_sales_revenue.value == pytest.approx(0.085379, rel=1e-4)
assert m.fs.costing_2.total_variable_OM_cost[0].value == pytest.approx(3.9969, rel=1e-4)
assert m.fs.costing_2.plant_overhead_cost[0].value == pytest.approx(1.0352, rel=1e-4)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.


Now, let's check some relevant costing model components to see how the custom model is integrated into the main costing equations:

In [12]:
for i in m.fs.costing_2.component_data_objects(Var, descend_into=True):
    if "custom" in str(i):
        print(i.name, "has a value of ", value(i))

print()

for i in m.fs.costing_2.component_data_objects(Constraint, descend_into=True):
    if "custom" in str(i.expr):
        print(i.expr)
        print()

fs.costing_2.custom_fixed_costs has a value of  1.4700714402797727e-05
fs.costing_2.custom_variable_costs has a value of  8.495651334916658e-06

fs.costing_2.total_BEC  ==  1e-06*(MUSD_2021/USD_2021)*fs.custom_vessel.costing.capital_cost + 1.1739346708671863e-06*(MUSD_2021/USD_2018)*fs.unit.costing.capital_cost

fs.costing_2.custom_fixed_costs  ==  1.0000000000000002e-06*(MUSD_2021/USD_2021)*fs.custom_vessel.costing.fixed_operating_cost

fs.costing_2.total_fixed_OM_cost  ==  fs.costing_2.annual_labor_cost + fs.costing_2.maintenance_and_material_cost + fs.costing_2.quality_assurance_and_control_cost + fs.costing_2.admin_and_support_labor_cost + fs.costing_2.sales_patenting_and_research_cost + fs.costing_2.property_taxes_and_insurance_cost + fs.costing_2.other_fixed_costs + fs.costing_2.watertap_fixed_costs + fs.costing_2.custom_fixed_costs

fs.costing_2.custom_variable_costs  ==  1.0000000000000002e-06*(MUSD_2021/USD_2021)*fs.custom_vessel.costing.variable_operating_cost

fs.costing_2.p

Although we didn't change anything about the `build_process_costs` call to note the custom model, the costing framework extracted the cost variables from the custom vessel. The `total_BEC` expression is the capital cost of our custom vessel, since that is the only unit in the flowsheet, and we can see that it accounts for the unit conversion between the model units of `MUSD_2022` and the flowsheet units of `MUSD_2021`. The operating cost equations also include the correct variables from our custom model, with the proper unit conversions.

# 4 Calculate Net Present Value and Taxes

Now, let's use the costing framework to calculate the net present value (NPV) of the process. The NPV represents the total value of the plant construction, operation, and production over the plant lifetime taking into account the changing worth of money over time. For example, distributed capital costs over multiple years or operating costs per year over the entire lifetime of a plant may grow with an escalation or inflation rate. The NPV provides a way to normalize costs over time to the current dollar year. Typical NPV components include capital cost, operating cost, revenue (this is a positive contribution, although we don't have one in the current example), and cash flows from capital loan repayment.

To calculate the NPV as part of the `build_process_costs` method, certain configuration arguments must be set when the flowsheet costing block is instantiated. Users may provide their own values instead to bypass this requirement (see Section 4.1) or link to the costing framework (see Section 4.2). Additionally, the framework supports tax calculations from process revenue and costs (see Section 4.3).

## 4.1 Calculate NPV From Fixed Inputs

The NPV calculation method may be utilized without calling the rest of the costing framework by setting fixed inputs for cost components when instantiating the costing block. This approach does not require any knowledge about the flowsheet, resources or product rates.

To begin, build the flowsheet costing block and specify the required cost components.

The total capital cost is 0.105 million USD, the total operating cost is the sum of the fixed and variable operating costs (9.173 million USD), and the total sales revenue is 0.085 million USD. The three arguments are required, and if one is set then all three must be set:

In [13]:
# Attach a flowsheet costing block with configuration arguments for NPV calculation
m.fs.costing_3 = QGESSCosting(
    discount_percentage=10,  # still a required argument, as before
    plant_lifetime=20,  # still a required argument, as before
    # set values for capex, opex, and revenue
    total_capital_cost=0.105,  # capex
    annual_operating_cost=9.173,  # opex
    annual_revenue=0.085,  # revenue
    cost_year="2021",  # tells method to return results in MUSD_2021
    # NPV arguments
    has_capital_expenditure_period=True,  # default is False, so need to set this to True
    # use defaults for all other arguments, don't need to set them here
)

Note the `total_capital_cost` above is the entire capital cost including the total BEC, installation costs, and any other plant costs. This is not the same as the `total_purchase_cost` passed to the `build_process_costs` method, which comprises the total BEC only.

Users must set the argument `cost_year` to define the currency units. The arguments `total_capital_cost`, `annual_operating_cost`, and `annual_revenue` may be passed as scalars, but can also be passed as `Expression`, `Var`, or `Param` objects with or without units of measurement. If there are units of measurements, the variables will be converted internally to the units set by `cost_year`.

Let's build the process costs. See below how we don't need to define anything else about the plant, or even set any other arguments, to calculate the NPV:

In [14]:
m.fs.costing_3.build_process_costs(
    fixed_OM=False,  # the required components don't exist, so we skip the fixed_OM method
    # by default, variable_OM is False
    calculate_NPV=True,
    CE_index_year="2021",
)

# Solve and display results (uncomment line to display)
solver.solve(m, tee=True)
# m.fs.costing_3.display() # uncomment to view results
assert m.fs.costing_3.npv.value == pytest.approx(-78.069, rel=1e-4)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resu

As shown in the output above (uncomment display command to see output), only the NPV-related attributes are created.

Checking some cost results, we see that the costing block stored the cost inputs locally:

In [15]:
print(value(m.fs.costing_3.capex))
print(value(m.fs.costing_3.opex))
print(value(m.fs.costing_3.revenue))

0.105
9.173
0.085


The report method skips the attributes that don't exist, so only the NPV is reported below:

In [16]:
m.fs.costing_3.report()


costing_3
------------------------------------------------------------------------------------
                        Value   
    Plant Cost Units   MUSD_2021
    Net Present Value    -78.069



## 4.2 Calculate NPV From Costing Block

The NPV calculations may also be coupled with the main costing framework, and automatically searches for relevant cost variables and expressions. To do this, we need to create our flowsheet costing block with several configuration arguments. Two arguments are required, the discount percentage defining cost projections over time and the plant lifetime in years. Many of the arguments have default values that are used if no other value is specified:

In [17]:
# Attach a flowsheet costing block with configuration arguments for NPV calculation
m.fs.costing_4 = QGESSCosting(
    # arguments for NPV
    discount_percentage=10,  # percentage; Rate of return used to discount future cash flows back to their present value
    plant_lifetime=20,  # years; Length of operating period
    has_capital_expenditure_period=True,  # True/false flag whether a capital expenditure period occurs, default False
    capital_expenditure_percentages=[
        10,
        60,
        30,
    ],  # A list of percentages that sum to 100 representing how capital costs are spread over a capital expenditure period
    capital_escalation_percentage=3.6,  # percentage; Rate at which capital costs escalate during the capital expenditure period, default 3.6%
    capital_loan_interest_percentage=6,  # percentage; Interest rate for capital equipment loan repayment, default 6%
    capital_loan_repayment_period=10,  # years; Length of loan repayment period, default 10 years
    debt_percentage_of_capex=50,  # percentage; Percentage of capex financed by debt, default 50%
    operating_inflation_percentage=3,  # percentage; Inflation rate for operating costs during the operating period, default 3%
    revenue_inflation_percentage=3,  # percentage; Inflation rate for revenue during the operating period, default 3%
)

If desired, users may set an argument `debt_expression` as an `Expression` to use instead of `debt_percentage_of_capex * capex`. This is an optional argument and will not be used here.

Now that the costing block has been created with the desired NPV-related arguments, we can set `calculate_NPV` in the `build_process_costs` method to add NPV calculations to the process costs. Because we did not specify any cost values in the costing block, the NPV method will look in the main costing block for required capital, operating, and revenue variables.

Now, let's build the process costs, adding a new argument `calculate_NPV` set to `True`:

In [18]:
# Build process costs
m.fs.costing_4.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    # treated stream is not saleable, so pass dummy inputs
    pure_product_output_rates={
        "treated_stream": m.fs.product[0] * 1.05
    },  # includes injected water
    mixed_product_output_rates={
        "treated_stream": m.fs.product[0] * 0
    },  # the product has 100% price realization, and is not considered a mixed product
    sale_prices={"treated_stream": 1 * pyunits.USD_2021 / pyunits.gal},
    variable_OM=True,
    resources=[
        "water",
        "chemicals",
        "waste",
    ],
    rates=[m.fs.water, m.fs.chemicals, m.fs.waste],
    prices={
        "chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon,
        "waste": 0.35 * pyunits.USD_2021 / pyunits.gallon,
    },
    watertap_blocks=[
        m.fs.unit,
    ],
    calculate_NPV=True,
    CE_index_year="2021",
)

QGESSCostingData.costing_initialization(m.fs.costing_4)
QGESSCostingData.initialize_fixed_OM_costs(m.fs.costing_4)
QGESSCostingData.initialize_variable_OM_costs(m.fs.costing_4)

# Solve and display results (uncomment line to display)
solver.solve(m, tee=True)
# m.fs.costing_4.display() # uncomment to view results
assert m.fs.costing_4.pv_capital_cost.value == pytest.approx(-0.00885464, rel=1e-4)
assert m.fs.costing_4.loan_debt.value == pytest.approx(0.00522987, rel=1e-4)
assert m.fs.costing_4.pv_loan_interest.value == pytest.approx(-0.000863716, rel=1e-4)
assert m.fs.costing_4.pv_operating_cost.value == pytest.approx(-78.7001, rel=1e-4)
assert m.fs.costing_4.pv_revenue.value == pytest.approx(0.7325064, rel=1e-4)
assert m.fs.costing_4.npv.value == pytest.approx(-77.9773, rel=1e-4)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resu

As shown above (uncomment display command to see output), we've added several new variables to the model; present values are assumed to be negative unless otherwise noted. The attribute `revenue` is a `Reference` to `total_sales_revenue` and it is required because Pyomo bars assigning an object to two places in the same model.

The NPV method solves for the following variables:

* `pv_capital_cost` - the present value of all capital costs over the plant lifetime
* `pv_operating_cost` - the present value of all operating costs over the plant lifetime
* `pv_revenue` - the present value of all revenue over the plant lifetime (positive value)
* `loan_debt` - principal loan taken out on capital
* `pv_loan_interest` - the present value of all loan interest payments over the plant lifetime
* `npv` - the net present value of the entire plant over its lifetime

and solves a new constraint corresponding to each new variable.

Let's look at the costing report:

In [19]:
m.fs.costing_4.report()


costing_4
------------------------------------------------------------------------------------
                                                                           Value   
    Plant Cost Units                                                      MUSD_2021
    Total Plant Cost                                                       0.010460
    Total Bare Erected Cost                                               0.0035218
    Total Installation Cost                                               0.0069380
    Total Other Plant Costs                                              1.0000e-12
    Total Fixed Operating & Maintenance Cost                                 5.1760
    Total Annual Operating Labor Cost                                        3.0730
    Total Annual Technical Labor Cost                                        1.1801
    Summation of Annual Labor Costs                                          4.2531
    Total Maintenance and Material Cost                         

We see that net present value is negative, which means that the plant is not economically viable and costs outweigh revenue over the plant lifetime.

Checking some cost results, we see that we see that that values very close to the prior case using fixed inputs, and that we obtain very similar NPV values:

In [20]:
print(value(m.fs.costing_4.capex))
print(value(m.fs.costing_4.opex))
print(value(m.fs.costing_4.revenue))

0.010459757918449586
9.172884943849951
0.08537719657954851


| Case | capex | opex | revenue | NPV |
| ---- | ----- | ---- | ------- | --- |
| Fixed Input | 0.105 | 9.173 | 0.085 | -78.069 | 
| Coupled With Costing | 0.010459757918449586 | 9.172884943849951 | 0.08537719657954851 | -77.977 |

## 4.3 Tax and Incentives Calculations

Suppose we know some information about the taxes and production incentives associated with constructing and operating this plant. We can define these properties, and the net present value method will automatically pick up these cash flows if present in the model. Users may include tax and incentives calculations by passing the following arguments to the `build_process_costs` method:

* `consider_taxes`: True/False flag for calculating net tax owed. Defaults to False.
* `income_tax_percentage`: combined federal and state income tax percentage, usually between 26 - 40%. Here, it defaults to 26%.
* `mineral_depletion_percentage`: fixed tax deduction percentage for mineral depletion based on the type of mineral recovered, defaults to 14% of gross income excluding royalties as reported in the UKy report.
* `production_incentive_percentage`: tax deduction percentage for producing critical minerals, defaults to 10% of total production cost (excludes cost of feedstock).
* `royalty_charge_percentage_of_revenue`: Percentage of revenue charged as royalties; defaults to 6.5% as reported in the UKy report.

Income tax is a negative cash flow and is a percentage of the total net sales revenue (total gross sales revenue minus total annual production cost). Similarly, royalties are a negative cash flow and are a fixed percentage of the total gross sales revenue. Income tax is typically set by local laws applying to the construction site, whereas royalties are charged by the land owner. The default charge rates are 26% income tax, per NETL QGESS guidelines, and 6.5% royalties, per the [University of Kentucky report](https://doi.org/10.2172/1569277).

Production incentives are a positive cash flow offsetting income tax based on the total annual production cost. Similarly, mineral depletion is a positive cash flow offsetting income tax based on total gross revenue excluding royalties. Production incentives are typically offered by governmental agencies to encourage production of certain material goods, whereas the mineral depletion charges allows plant operators to recover a portion of the expense associated with the reduced site value from extracting materials. The default rates are 10% production incentives, per the 45X mineral production credit, and 14% mineral depletion, per the [University of Kentucky report](https://doi.org/10.2172/1569277).

Additionally, the net present value method will create and solve for a variable `pv_taxes` for the present value of taxes projected over the plant lifetime.

The University of Kentucky report and case study are discussed further in the [UKy Flowsheet](uky_flowsheet-solution.ipynb) tutorial and with costing in the [UKy Flowsheet with Costing](costing_uky_flowsheet-solution.ipynb) tutorial.

As our current scenario involves brine nanofiltration and not mineral processing, the mineral depletion rate does not apply and is set to zero; however, the process still qualifies for a production incentive and this term is thus included.

The code below demonstrates including tax calculations in overnight cost and net present value calculations:

In [21]:
# Attach a new costing block to the flowsheet
m.fs.costing_5 = QGESSCosting(
    # arguments for NPV
    discount_percentage=10,  # percent
    plant_lifetime=20,  # years
    has_capital_expenditure_period=True,
    capital_expenditure_percentages=[10, 60, 30],
    capital_escalation_percentage=3.6,
    capital_loan_interest_percentage=6,
    capital_loan_repayment_period=10,
    debt_percentage_of_capex=50,
    operating_inflation_percentage=3,
    revenue_inflation_percentage=3,
)

# Build process costs
m.fs.costing_5.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    # treated stream is not saleable, so pass dummy inputs
    pure_product_output_rates={
        "treated_stream": m.fs.product[0] * 1.05
    },  # includes injected water
    mixed_product_output_rates={
        "treated_stream": m.fs.product[0] * 0
    },  # the product has 100% price realization, and is not considered a mixed product
    sale_prices={"treated_stream": 1 * pyunits.USD_2021 / pyunits.gal},
    variable_OM=True,
    resources=[
        "water",
        "chemicals",
        "waste",
    ],
    rates=[m.fs.water, m.fs.chemicals, m.fs.waste],
    prices={
        "chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon,
        "waste": 0.35 * pyunits.USD_2021 / pyunits.gallon,
    },
    watertap_blocks=[
        m.fs.unit,
    ],
    consider_taxes=True,
    income_tax_percentage=26,
    mineral_depletion_percentage=0,
    production_incentive_percentage=10,
    royalty_charge_percentage_of_revenue=6.5,
    calculate_NPV=True,
    CE_index_year="2021",
)

QGESSCostingData.costing_initialization(m.fs.costing_5)
QGESSCostingData.initialize_fixed_OM_costs(m.fs.costing_5)
QGESSCostingData.initialize_variable_OM_costs(m.fs.costing_5)
solver.solve(m, tee=True)
# m.fs.costing_5.display() # uncomment to view results
assert value(m.fs.costing_5.royalty_charge) == pytest.approx(0.0055495, rel=1e-4)
assert value(m.fs.costing_5.mineral_depletion_charge) == pytest.approx(0, abs=1e-8)
assert value(m.fs.costing_5.production_incentive_charge) == pytest.approx(
    0.91741, rel=1e-4
)
assert m.fs.costing_5.income_tax.value == pytest.approx(-2.3631, rel=1e-4)
assert m.fs.costing_5.additional_tax_owed.value == pytest.approx(0, abs=1e-8)
assert m.fs.costing_5.additional_tax_credit.value == pytest.approx(0, abs=1e-8)
assert m.fs.costing_5.net_tax_owed.value == pytest.approx(0, abs=1e-8)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resu

In [22]:
m.fs.costing_5.report()


costing_5
------------------------------------------------------------------------------------
                                                                           Value   
    Plant Cost Units                                                      MUSD_2021
    Total Plant Cost                                                       0.010460
    Total Bare Erected Cost                                               0.0035218
    Total Installation Cost                                               0.0069380
    Total Other Plant Costs                                              1.0000e-12
    Total Fixed Operating & Maintenance Cost                                 5.1760
    Total Annual Operating Labor Cost                                        3.0730
    Total Annual Technical Labor Cost                                        1.1801
    Summation of Annual Labor Costs                                          4.2531
    Total Maintenance and Material Cost                         

In the report above, the tax components are now included. Processes in which the production costs are large and the sales revenue is small could have a large production credit and a small combined income tax and royalties cash flow, yielding a negative tax. In the case of a calculated negative tax, the tax contribution is bounded by an internal model variable `min_net_tax_owed` that defaults to 0. Users may override this enforce a minimum tax owed, or provide an allowance for a negative tax. The minimum tax value is changed below and the resulting net tax owed and NPV are presented:

In [23]:
print("Net Taxed Owed: ", value(m.fs.costing_5.net_tax_owed))
print("NPV: ", value(m.fs.costing_5.npv))

print("\nEnforce a minimum tax of $3 million per year")
m.fs.costing_5.min_net_tax_owed.fix(3)
solver.solve(m, tee=False)
print("Net Taxed Owed: ", value(m.fs.costing_5.net_tax_owed))
print("NPV: ", value(m.fs.costing_5.npv))

print(
    "\nArtificially increase the tax credit and allow a negative tax up to -$1 million per year"
)
m.fs.costing_5.additional_tax_credit.fix(5)
m.fs.costing_5.min_net_tax_owed.fix(-1)
solver.solve(m, tee=False)
print("Net Taxed Owed: ", value(m.fs.costing_5.net_tax_owed))
print("NPV: ", value(m.fs.costing_5.npv))

Net Taxed Owed:  7.643892704437963e-10
NPV:  -77.97734966988045

Enforce a minimum tax of $3 million per year
component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Net Taxed Owed:  3.000000000396878
NPV:  -97.16645263036031

Artificially increase the tax credit and allow a negative tax up to -$1 million per year
component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Net Taxed Owed:  -0.9999999996557521
NPV:  -71.58102814994122


# 5 Economy of Numbers

The costing framework supports calculation of process cost savings using economy of numbers, which estimates future profitability of novel equipment and cost savings due to experiential learning. Let's suppose that our brine nanofiltration process becomes more efficient over time as the technology develops; we want to see how much the cost reduces due to knowledge gain over time.

## 5.1 Mathematical Definition

The cost of manufacturing a piece of equipment tends to decline as the production quantity increases (economy of scales), but also as new generations of that equipment are developed leading to increases in technical knowledge (economy of numbers). New equipment are often denoted as First-of-a-Kind (FOAK) equipment while later generations with improved knowledge are denoted as Nth-of-a-Kind (NOAK) equipment.

The following equations are taken from Faber G, Ruttinger A, Strunge T, Langhorst T, Zimmermann A, van der Hulst M, Bensebaa F, Moni S and Tao L (2022) Adapting Technology Learning Curves for Prospective Techno-Economic and Life Cycle Assessments of Emerging Carbon Capture and Utilization Pathways. Front. Clim. 4:820261. doi: 10.3389/fclim.2022.820261:

$Y = A * x^b$

$b = - \frac{\log_{10}(1-R)}{\log_{10}(2)}$

where $Y$ is the NOAK cost, $A$ is the FOAK cost, $x$ is the cumulative number of units (or generations of equipment), $b$ is the learning rate exponent, and $R$ is the learning rate. The learning rate is a matter of engineering knowledge for various technologies and plant configurations, and range between 0.01 to 0.1 depending on the level of maturity (experimental, growing, proven, etc.). More information on typical learning rates may be found at Rubin, E. S., Mantripragada, H., and Zhai, H., "An Assessment of the NETL Cost Estimation Methodology". Department of Engineering and Public Policy, Carnegie Mellon University, Pittsburgh, PA (2016). p. 31, Fig. 6-4.

## 5.2 Usage Example

Our first-generation brine nanofiltration resulted in a total (installed) plant cost of $0.10459 million. The item has undergone development and is a 5th-generation design. The learning rate for the technology is 0.05.

The following code may be used to predict the NOAK cost:

In [24]:
# Calculate the economy of numbers benefit
QGESSCostingData.economy_of_numbers(
    m.fs.costing_5,
    cum_num_units=5,
    cost_FOAK=m.fs.costing_5.total_plant_cost,
    CE_index_year="2021",
    learning_rate=0.05,
)

solver = get_solver()
results = solver.solve(m, tee=True)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resu

In [25]:
# display results (uncomment line to display)
m.fs.costing_5.display()
assert m.fs.costing_5.cost_NOAK.value == pytest.approx(0.00928533, rel=1e-4)

Block fs.costing_5

  Variables:
    total_BEC : Total TPC
        Size=1, Index=None, Units=MUSD_2021
        Key  : Lower : Value                : Upper : Fixed : Stale : Domain
        None :     0 : 0.003521804012602152 :  None : False : False :  Reals
    total_installation_cost : Total installation cost
        Size=1, Index=None, Units=MUSD_2021
        Key  : Lower : Value                : Upper : Fixed : Stale : Domain
        None :     0 : 0.006937953904826436 :  None : False : False :  Reals
    total_plant_cost : Total plant cost
        Size=1, Index=None, Units=MUSD_2021
        Key  : Lower : Value                : Upper : Fixed : Stale : Domain
        None :     0 : 0.010459757918428758 :  None : False : False :  Reals
    other_plant_costs : Additional plant costs
        Size=1, Index=None, Units=MUSD_2021
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 1e-12 :  None :  True :  True :  Reals
    annual_operating_labor_cost : Annu

As shown in the output above (uncomment display command to see output), the 5th-generation equipment cost is &#0036; 0.09285 million USD, which corresponds to a cost reduction of 11.2% relative to the first-generation cost of &#0036; 0.10459 million USD.

# 6 Costing Bounding

Finally, the costing framework supports calculating costing bounds based on the feedstock capacity and grade of a process. This method applies to various types of mineral processing, not brine nanofiltration; as such, a toy example will be used here rather than the brine nanofiltration process

Like the fixed input NPV method in section 4 and the economy of numbers method in Section 5, the costing bounding method is independent of the main costing framework. The bound equations derive from Fritz, A.G., Tarka, T.J. & Mauter, M.S. Assessing the economic viability of unconventional rare earth element feedstocks. Nat Sustain 6, 1103–1112 (2023). https://doi.org/10.1038/s41893-023-01145-1. The study establishes a database of capital and operating expenses for REE recovery and market prices of saleable mineral products, and develops surrogates models to predict the maximum cost threshold of specific process types to determine the economic viability. Thresholds are calculated using a power law:

$ C_{threshold} (2022 USD / kg REE) = \alpha * x^\beta$

where $C_{threshold}$ is the cost threshold, $x$ is a known quantity about the process, and $\alpha$ and $\beta$ are empirical coefficients. A dictionary of coefficients were regressed from process data for a range of process types with $x$ as the total feedstock REE:

$ x = F_{grade}(unitless) * F_{capacity}(tonnes)$

The values [$\alpha$, $\beta$] are available in the public paper and the costing framework code, and are presented below:

* "Total Capital": [81, -0.46]
* "Total Operating": [27, -0.087]
* "Beneficiation": [2.7, -0.15]
* "Beneficiation, Chemical Extraction, Enrichment and Separation": [22, -0.059]
* "Chemical Extraction": [40, -0.46]
* "Chemical Extraction, Enrichment and Separation": [15, -0.19]
* "Enrichment and Separation": [6.7, -0.16]
* "Mining": [25, -0.32]

## 6.1 Quantifying Threshold Bounds

To quantify uncertainty in the threshold estimation, a 95% confidence interval was applied to the values to produce upper and lower bound predictions for the cost threshold per the following equation:

$C_{threshold}^{\pm} (2022 USD / kg REE) = ({\alpha} \pm {\alpha}_{0.95}) * x^{\beta \pm {\beta}_{0.95}}$

where $C_{threshold}^{-}$ corresponds to the 5% confidence bound and $C_{threshold}^{+}$ corresponds to the 95% confidence bound.

The following values, available in the costing framework code, were produced for [$\alpha$, ${\alpha}_{0.95}$, $\beta$, ${\beta}_{0.95}$]:

* "Total Capital": [81, 1.4, -0.46, 0.063]
* "Total Operating": [27, 0.87, -0.087, 0.038]
* "Beneficiation": [2.7, 1.3, -0.15, 0.062]
* "Beneficiation, Chemical Extraction, Enrichment and Separation": [22, 1.28,  -0.059, 0.046]
* "Chemical Extraction, Enrichment and Separation": [15, 15, -0.19, 0.28]
* "Enrichment and Separation": [6.7, 2.8, -0.16, 0.11]
* "Mining": [25, 2.5, -0.32, 0.095]

## 6.2 Usage Example

The method itself is straightforward to implement if the required inputs are provided. Suppose we have a process with a feed input rate of 500 U.S. ton of REE per hour with an REE grade of 356.64 ppm. We can call the costing bounding method, which will print the calculated bound values after solving:

In [26]:
# Create a Concrete Model as the top level object
# m = ConcreteModel()

# Add a flowsheet object to the model
# m.fs = FlowsheetBlock(dynamic=False)

# Attach a flowsheet costing block with configuration arguments for NPV calculation
m.fs.costing_6 = QGESSCosting()

# Define inputs
m.fs.feed_input = Var(initialize=500, units=pyunits.ton / pyunits.hr)
m.fs.feed_grade = Var(initialize=356.64, units=pyunits.ppm)

# Define annual operating hours
hours_per_shift = 8
shifts_per_day = 3
operating_days_per_year = 336

m.fs.annual_operating_hours = Param(
    initialize=hours_per_shift * shifts_per_day * operating_days_per_year,
    mutable=True,
    units=pyunits.hours / pyunits.a,  # pyunits.a is "annum", equivalent to pyunits.year
)

# Define plant lifetime
m.fs.plant_lifetime = Param(initialize=20, mutable=True, units=pyunits.a)

# Call costing bound method
CE_index_year = "2021"
QGESSCostingData.calculate_REE_costing_bounds(
    b=m.fs.costing_6,  # block to attach costing bound variables and constraints to
    capacity=m.fs.feed_input
    * m.fs.annual_operating_hours
    * m.fs.plant_lifetime,  # lifetime capacity
    grade=m.fs.feed_grade,  # feed grade
    CE_index_year=CE_index_year,  # project dollar year
)

2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'capacity' created as attribute of fs.costing_6
2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'grade' created as attribute of fs.costing_6
2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'costing_lower_bound' created as attribute of fs.costing_6
2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'costing_upper_bound' created as attribute of fs.costing_6
2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New constraint 'costing_lower_bounding_eq' created as attribute of fs.costing_6
2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New constraint 'costing_upper_bounding_eq' created as attribute of fs.costing_6
2025-08-19 08:02:04 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: 

Printing calculated costing bounds for processes:
Total Capital : [ 0.3

The results above were calculated directly from the inputs, and in this case are the correct values. This is not the case for flowsheets in general, as we did not solve the flowsheet yet.

Solving and printing the bound values, we confirm the same results:

In [27]:
# Solve and display results
solver.solve(m, tee=True)

processes = [
    "Total Capital",
    "Total Operating",
    "Beneficiation",
    "Beneficiation, Chemical Extraction, Enrichment and Separation",
    "Chemical Extraction",
    "Chemical Extraction, Enrichment and Separation",
    "Enrichment and Separation",
    "Mining",
]

print("\n\nPrinting calculated costing bounds for processes:")
for p in processes:
    print(
        p,
        ": [",
        value(m.fs.costing_6.costing_lower_bound[p]),
        ", ",
        value(m.fs.costing_6.costing_upper_bound[p]),
        "]",
        getattr(pyunits, "USD_" + CE_index_year),
        "/kg",
    )

# check a few entries
assert m.fs.costing_6.costing_lower_bound["Total Capital"].value == pytest.approx(
    0.338407, rel=1e-4
)
assert m.fs.costing_6.costing_upper_bound["Total Capital"].value == pytest.approx(
    1.26162, rel=1e-4
)
assert m.fs.costing_6.costing_lower_bound["Total Operating"].value == pytest.approx(
    6.35950, rel=1e-4
)
assert m.fs.costing_6.costing_upper_bound["Total Operating"].value == pytest.approx(
    14.6917, rel=1e-4
)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resu

# Summary

The advanced method demonstrated here expand upon the capabilities of the REE Costing Framework. Syntax for the example WaterTAP and custom models may be applied to any imported WaterTAP or custom model, respectively, and it is recommended that additional unit operation specificaitions are placed prior to calling the `build_process_costs`. The NPV calculations must be set up when instantiating the flowsheet costing block itself, as shown in this notebook, while the costing bounding equations are entirely independent of the capital and operating cost equations and may be added at any point while building the flowsheet.